# Let’s embrace WebAssembly!

[Presentation made at EuroPython 2018 - Edinburgh (by Almar Klein)](https://www.youtube.com/watch?v=u2kKxmb9BWs&index=18&list=PL8uoeex94UhFrNUV2m5MigREebUms39U5)

This Notebook is:
- a compacted version of original https://github.com/almarklein/rocket_rust_py
- best seen in RISE mode: <center><img src='images/start_rise_mode.png' width=800px></center>




# WASM has a compact binary format



#### And a human readable counterpart:

```wasm
(module
    (type $print (func (param i32))
    (func $main
        (i32.const 42)
        (call $print)
    )
    (start $main)
)
```


# It's safe


Because browsers.

# Use case 1: Compile a subset of Python to WASM

<center><img src='images/pysnippet_to_wasm.png' width=800px></center>

In [1]:
# in RISE mode, click <Shift>+<Enter> to execut a cell
def find_prime(nth):
    n = 0
    i = -1       
    while n < nth:
        i = i + 1        
        if i <= 1:
            continue  # nope
        elif i == 2:
            n = n + 1
        else:
            gotit = 1
            for j in range(2,  i//2+1):
                if i % j == 0:
                    gotit = 0
                    break
            if gotit == 1:
                n = n + 1    
    return i

In [2]:
%time find_prime(1000)

Wall time: 364 ms


7919

# Compiling 'find_prime()' to  WASM


Note: 

* The python-to-wasm compiler is just a POC!
* Assumes a (reliable) wasm-to-native compiler

In [3]:
# in RISE mode, click <Shift>+<Enter> to execut a cell
from ppci import wasm

In [4]:
from ppci.lang.python import python_to_wasm

def main():
    print(find_prime(1000))

m = python_to_wasm(main, find_prime)

In [5]:
# WASM (somewhat) readable machine code
m.show()

(module
    (type $0 (func (param f64)))
    (type $1 (func))
    (type $2 (func (param f64) (result f64)))
    (import "env" "f64_print" (func $print (type $0)))
    (export "main" (func $main))
    (export "find_prime" (func $find_prime))
    (start $main)
    (func $main (type $1)
        (f64.const 1000.0)
        (call $find_prime)
        (call $print)
    )
    (func $find_prime (type $2) (local f64 f64 f64 f64 f64 f64 f64)
        (f64.const 0.0)
        (set_local 1)
        (f64.const 1.0)
        (f64.neg)
        (set_local 2)
        (block)
            (loop)
                (get_local 2)
                (f64.const 1.0)
                (f64.add)
                (set_local 2)
                (get_local 2)
                (f64.const 1.0)
                (f64.le)
                (if)
                    (br 1)
                (else)
                    (get_local 2)
                    (f64.const 2.0)
                    (f64.eq)
                    (if)
                    

In [6]:
# WASM binary format
m.show_bytes()

00000000  00 61 73 6d 01 00 00 00 01 0d 03 60 01 7c 00 60  .asm.......`.|.`
00000010  00 00 60 01 7c 01 7c 02 11 01 03 65 6e 76 09 66  ..`.|.|....env.f
00000020  36 34 5f 70 72 69 6e 74 00 00 03 03 02 01 02 07  64_print........
00000030  15 02 04 6d 61 69 6e 00 01 0a 66 69 6e 64 5f 70  ...main...find_p
00000040  72 69 6d 65 00 02 08 01 01 0a 93 02 02 0f 00 44  rime...........D
00000050  00 00 00 00 00 40 8f 40 10 02 10 00 0b 80 02 01  .....@.@........
00000060  07 7c 44 00 00 00 00 00 00 00 00 21 01 44 00 00  .|D........!.D..
00000070  00 00 00 00 f0 3f 9a 21 02 02 40 03 40 20 02 44  .....?.!..@.@..D
00000080  00 00 00 00 00 00 f0 3f a0 21 02 20 02 44 00 00  .......?.!...D..
00000090  00 00 00 00 f0 3f 65 04 40 0c 01 05 20 02 44 00  .....?e.@.....D.
000000a0  00 00 00 00 00 00 40 61 04 40 20 01 44 00 00 00  ......@a.@..D...
000000b0  00 00 00 f0 3f a0 21 01 05 44 00 00 00 00 00 00  ....?.!..D......
000000c0  f0 3f 21 03 44 00 00 00 00 00 00 00 40 20 02 44  .?!.D.......@..D
000000d0  00

In [7]:
# WASM interface
m.show_interface()

Imports:
  env.f64_print:     [f64] -> []
Exports:
  main:              [] -> []
  find_prime:        [f64] -> [f64]


# Run in Browser 


In [8]:
wasm.run_wasm_in_notebook(m)

<IPython.core.display.Javascript object>

# Run in NodeJS


In [9]:
wasm.run_wasm_in_node(m)

Hello from Nodejs!
Compiling wasm module
Initializing wasm module
Result:
7919
(in 0.01830307301133871 s)


'7919\n(in 0.01830307301133871 s)'

# Compile 'find_prime()' to WASM then to Machine code

In [10]:
@wasm.wasmify
def find_prime2(nth):
    n = 0
    i = -1       
    while n < nth:
        i = i + 1        
        if i <= 1:
            continue  # nope
        elif i == 2:
            n = n + 1
        else:
            gotit = 1
            for j in range(2,  i//2+1):
                if i % j == 0:
                    gotit = 0
                    break
            if gotit == 1:
                n = n + 1    
    return i

In [12]:
%time find_prime2(1000)

Wall time: 499 µs


0.0

# Use case 2: Python as a platform to bind and run WASM modules
    
... and allow that code to call into Python functions

<center><img src='images/py_as_platform.png' width=700px></center>

# the Rocket game from [github.com/aochagavia](https://github.com/aochagavia/rocket_wasm)

<center>
    <!-- <a href='https://thread-safe.nl/rocket/' target='new'> -->
    <a href='rocket.html' target='new'>
    <img src='images/github_rocket_wasm.png' width=900>
    </a>
</center>

# The rocket game is in a single binary WASM file (58 KB)

<center>    
    <img src='images/github_rocket_wasm_html.png' width=600>
</center>
<center>
<img src='images/rocketgame.png' width=800>
</center>

In [ ]:
from ppci import wasm

m = wasm.Module(open(r'wasm/rocket.wasm', 'rb'))
m

In [13]:
m.show_interface()

Imports:
  env.f64_print:     [f64] -> []
Exports:
  main:              [] -> []
  find_prime:        [f64] -> [f64]


<center>
<img src='images/github_rocket_wasm_js.png' width=1000>
</center>

<center>
<img src='images/rocket_in_js.png' width=800>
</center>

<center>
<img src='images/rocket_in_py.png' width=800>
</center>

In [ ]:
class PythonRocketGame:
    
    # ...
    
    def wasm_sin(self, a:float) -> float:
        return math.sin(a)
    
    def wasm_cos(self, a:float) -> float:
        return math.cos(a)
    
    def wasm_Math_atan(self, a:float) -> float:
        return math.atan(a)
    
    def wasm_clear_screen(self) -> None:
        # ...
    
    def wasm_draw_bullet(self, x:float, y:float) -> None:
        # ...
    
    def wasm_draw_enemy(self, x:float, y:float) -> None:
        # ...
    
    def wasm_draw_particle(self, x:float, y:float, a:float) -> None:
        # ...
    
    def wasm_draw_player(self, x:float, y:float, a:float) -> None:
       # ...
    
    def wasm_draw_score(self, score:float) -> None:
        # ...

# Run Rocket in Python with Qt



In [14]:
from rocket_qt import QtRocketGame
game = QtRocketGame()

In [15]:
# you may have to switch to the QT window appearing on the side of your browser session
game.run()

# This game is not that hard to play ...

## Let's make an AI!

#### Game in rust compiled in WASM
#### AI in C compiled in WASM
#### Glue in Python

In [16]:
# let's write the AI in C
print(open('wasm/ai2.c', 'rt').read())

#include <stdlib.h>
#include <math.h>

/* Imports */
void toggle_shoot(int b);
void toggle_turn_left(int b);
void toggle_turn_right(int b);
float Math_atan(float v);
//void debug(float a, float b);

const float pi = 3.14159265358979;

float playerX = 0.0;
float playerY = 0.0;
float playerA = 0.0;

float nearestEnemyX = 0.0;
float nearestEnemyY = 0.0;
float nearestEnemyD = 99999999999999999.0;

void clear_screen() { // reset nearest enemy
  nearestEnemyX = 0.0;
  nearestEnemyY = 0.0;
  nearestEnemyD = 99999999999999999.0;
}

void draw_player(float x, float y, float a) {
  playerX = x;
  playerY = y;
  while (a > 2*pi)  a -= 2*pi;
  while (a < 0)  a += 2*pi2;
  playerA = a;
}

void draw_enemy(float x, float y) {
  float d = (x - playerX) * (x - playerX) + (y - playerY) * (y - playerY);
  if (d < nearestEnemyD) {
    nearestEnemyX = x;
    nearestEnemyY = y;
    nearestEnemyD = d;
  }
}

void update() {
  float enemyA = Math_atan((nearestEnemyY - playerY) / (nearestEnemyX - playerX));
  


In [17]:
# use https://wasdk.github.io/WasmFiddle/ to convert ai.c in ai2.wasm
from ppci import wasm
ai2 = wasm.Module(open('wasm/ai2.wasm', 'rb'))

In [18]:
ai2.show_interface()

Imports:
  env.Math_atan:     [f32] -> [f32]
  env.toggle_shoot:  [i32] -> []
  env.toggle_turn_left: [i32] -> []
  env.toggle_turn_right: [i32] -> []
Exports:
  memory:            "memory"
  clear_screen:      [] -> []
  draw_player:       [f32, f32, f32] -> []
  draw_enemy:        [f32, f32] -> []
  update:            [] -> []


In [19]:
from rocket_ai import AiRocketGame
game = AiRocketGame(ai2)
game.run()

0


OSError: exception: access violation reading 0x0953FFFF


# Wrapping up ...

## WASM is coming, and its awesome!


<img src='images/wasm.png' width=150 align=right>

* Open, low-level, fast, compact and safe
* Already works in most browsers
* Not limited to the web



<img src='images/python.png' width=150 align=right>

## We Pythonista's should embrace it!

* E.g. run a Python VM in the browser
* E.g. compile subset of Python to fast, crossplatform code
* E.g. use Python as a platform to bind and execute WASM modules

## Thanks!